In [ ]:
import os
print("Current working directory:", os.getcwd())

In [2]:
"""
Run Simple Parallel Scan for Test Bar

This script runs the test bar scan across multiple exchanges in parallel
using a simplified parallel scanning approach that avoids console output issues.
"""

import asyncio
import sys
import os
import logging

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(message)s')

# Add project directory to path
project_dir = os.path.join(os.getcwd(), "Project")
sys.path.insert(0, project_dir)
print(f"✓ Added {project_dir} to sys.path")

# Add current directory to path
sys.path.append(os.getcwd())
print(f"✓ Added {os.getcwd()} to sys.path")

# Import the simple parallel scanner
from run_parallel_scanner import run_parallel_exchanges, run_parallel_multi_timeframes_all_exchanges, sf_exchanges_1w
from scanner.main import kline_cache

# Define exchanges
futures_exchanges = ["binance_futures", "bybit_futures", "mexc_futures", "gateio_futures"]
spot_exchanges = ["binance_spot", "bybit_spot", "kucoin_spot", "mexc_spot", "gateio_spot"]
spot_exchanges_1w = ["binance_spot", "bybit_spot", "gateio_spot"] + sf_exchanges_1w
fast_exchanges = ["binance_futures", "binance_spot", "bybit_spot", "gateio_spot"]
slow_exchanges = ["kucoin_spot", "mexc_spot", "mexc_futures"]

async def main():
    # Clear cache for fresh data
    kline_cache.clear()
    
    """    
    # Run parallel scan for test bar strategy on spot exchanges
    result = await run_parallel_exchanges(
        timeframe="4h",                    # Example timeframe
        strategies=["hbs_breakout", "test_bar", "consolidation_breakout", "sma50_breakout", "trend_breakout", "pin_up"],
        # strategies=["reversal_bar"],       
        exchanges=spot_exchanges,          # Spot exchanges to scan
        users=["default"],                 # Recipients for Telegram notifications
        send_telegram=True,                # Enable Telegram notifications
        min_volume_usd=None,               # Use default volume threshold
        save_to_csv=True                   # Enable saving to CSV
    )
    """
    # Run multi-timeframe parallel scan
    result = await run_parallel_multi_timeframes_all_exchanges(
        timeframes=["4h"],     # Multiple timeframes0
        strategies= ["vs_wakeup"], #["confluence", "consolidation_breakout", "channel_breakout", "sma50_breakout", "loaded_bar", "pin_up", "trend_breakout"], #"engulfingReversal", "vs_wakeup],        # Strategies to scan
        exchanges= fast_exchanges,          # Exchanges to scan
        users=["default"],                 # Recipients for notifications
        send_telegram=True,                # Enable notifications
        min_volume_usd=None,               # Use default volume threshold
        save_to_csv=False                   # Enable saving to CSV
    )
     #"""
    
    print("Scan completed!")
    return result

if __name__ == "__main__":
    asyncio.run(main())


  RUNNING PARALLEL MULTI-TIMEFRAME SCAN ON ALL EXCHANGES

• Exchanges: binance_futures, binance_spot, bybit_spot, gateio_spot
• Timeframes: 4h
• Strategies: vs_wakeup
• Notifications: Enabled
• Recipients: default
• Save to CSV: Disabled
• Start time: 07:41:58

Fetching market data...

Processing timeframe: 4h
4h: 4 FAST, 0 SLOW exchanges

  PHASE: FAST 4h (4 exchanges)

[07:41:58] Starting scan on binance_futures for 4h timeframe...
[07:41:58] Starting scan on gateio_spot for 4h timeframe...
[07:41:58] Starting scan on binance_spot for 4h timeframe...


✓ Added /home/jovyan/work/Crypto/sevenfigures-bot/hbs_2025/Project/Project to sys.path
✓ Added /home/jovyan/work/Crypto/sevenfigures-bot/hbs_2025/Project to sys.path


[07:41:58] Starting scan on bybit_spot for 4h timeframe...
Found 493 markets on Binance Futures for 4h timeframe
Processing 493 symbols with parallel strategies (batch size: 25)
Found 509 markets on Bybit for 4h timeframe
Processing 509 symbols with parallel strategies (batch size: 25)
Found 418 markets on Binance Spot for 4h timeframe
Processing 418 symbols with parallel strategies (batch size: 25)
Found 2198 markets on Gateio for 4h timeframe
Processing 2198 symbols with parallel strategies (batch size: 25)
[07:42:33] ✓ Completed binance_spot scan: 0 signals found
[07:42:36] ✓ Completed bybit_spot scan: 0 signals found
[07:42:37] ✓ Completed binance_futures scan: 0 signals found
[07:44:15] ✓ Completed gateio_spot scan: 0 signals found

  PHASE: SLOW 4h (0 exchanges)

No exchanges in this phase.

  PARALLEL MULTI-TIMEFRAME MULTI-EXCHANGE SCAN RESULTS

Total signals found across all exchanges and timeframes: 0
Start time: 07:41:58
End time: 07:44:15
Duration: 0:02:17


Scan completed!


In [ ]:
#Verify OHLCV data from SF

from exchanges.sf_pairs_service import SFPairsService
import pandas as pd

sf_service = SFPairsService()

symbol = "BTC"
quote = "USDT"
exchange = "Kucoin"   # or "Mexc"
timeframe = "1w"      # weekly
limit = 2             # how many candles to fetch

# Fetch OHLCV data
raw_data = sf_service.get_ohlcv_for_pair(symbol, quote, exchange, timeframe, limit)

# Convert to DataFrame
df = pd.DataFrame(raw_data)

# Normalize datetime
if 'datetime' in df.columns:
    df['datetime'] = pd.to_datetime(df['datetime'])
    df = df.set_index('datetime')
elif 'time' in df.columns:
    df['time'] = pd.to_datetime(df['time'], unit='ms')
    df = df.set_index('time')

# Keep only required columns
df = df[['open', 'high', 'low', 'close', 'volume']].astype(float)

# Print with timeframe in console
print(f"\n📊 {symbol}/{quote} OHLCV ({timeframe}) on {exchange}")
print(df.tail(1))  # last row = current bar


In [ ]:
# Binance BTC dominated pairs - Confluence Scanner with Direct API

from telegram.ext import Application
import logging
import pandas as pd
import numpy as np
import asyncio
import nest_asyncio
from datetime import datetime
from tqdm.asyncio import tqdm
import sys
import os
import html
import aiohttp
import time

# Add project path
project_dir = os.path.join(os.getcwd(), "Project")
sys.path.insert(0, project_dir)

from custom_strategies import detect_confluence

class BinanceBTCConfluenceScanner:
    def __init__(self, telegram_token, telegram_chat_id, timeframe, offset=1):
        self.telegram_token = telegram_token
        self.telegram_chat_id = telegram_chat_id
        self.telegram_app = None
        self.exchange = "Binance"
        self.timeframe = timeframe
        self.quote_currency = "BTC"
        self.offset = offset
        self.base_url = "https://api.binance.com"
        self.session = None
        
        # Binance timeframe mapping - All available intervals
        self.timeframe_map = {
            # Minutes
            "1m": "1m",
            "3m": "3m", 
            "5m": "5m",
            "15m": "15m",
            "30m": "30m",
            # Hours
            "1h": "1h",
            "2h": "2h",
            "4h": "4h",
            "6h": "6h",
            "8h": "8h",
            "12h": "12h",
            # Days
            "1d": "1d",
            "2d": "2d",
            "3d": "3d",
            # Weeks/Months
            "1w": "1w",
            "1M": "1M"
        }
        
    async def init_session(self):
        """Initialize aiohttp session"""
        if self.session is None:
            self.session = aiohttp.ClientSession()
            
    async def close_session(self):
        """Close aiohttp session"""
        if self.session:
            await self.session.close()
            self.session = None
        
    async def init_telegram(self):
        if self.telegram_app is None:
            self.telegram_app = Application.builder().token(self.telegram_token).build()

    async def get_btc_pairs(self):
        """Get all BTC trading pairs from Binance"""
        await self.init_session()
        
        try:
            url = f"{self.base_url}/api/v3/exchangeInfo"
            async with self.session.get(url) as response:
                if response.status == 200:
                    data = await response.json()
                    
                    # Filter for BTC pairs that are actively trading
                    btc_pairs = []
                    for symbol_info in data['symbols']:
                        if (symbol_info['quoteAsset'] == 'BTC' and 
                            symbol_info['status'] == 'TRADING' and
                            symbol_info['isSpotTradingAllowed']):
                            
                            btc_pairs.append({
                                'symbol': symbol_info['symbol'],
                                'baseAsset': symbol_info['baseAsset'],
                                'quoteAsset': symbol_info['quoteAsset']
                            })
                    
                    return btc_pairs
                else:
                    logging.error(f"Error fetching exchange info: {response.status}")
                    return []
        except Exception as e:
            logging.error(f"Error fetching BTC pairs: {str(e)}")
            return []

    async def get_klines(self, symbol, interval, limit=100):
        """Get kline/candlestick data from Binance"""
        await self.init_session()
        
        try:
            url = f"{self.base_url}/api/v3/klines"
            params = {
                'symbol': symbol,
                'interval': interval,
                'limit': limit
            }
            
            async with self.session.get(url, params=params) as response:
                if response.status == 200:
                    data = await response.json()
                    
                    # Convert to DataFrame
                    df = pd.DataFrame(data, columns=[
                        'open_time', 'open', 'high', 'low', 'close', 'volume',
                        'close_time', 'quote_asset_volume', 'number_of_trades',
                        'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'
                    ])
                    
                    # Convert timestamps to datetime
                    df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
                    df['close_time'] = pd.to_datetime(df['close_time'], unit='ms')
                    
                    # Convert OHLCV to numeric
                    for col in ['open', 'high', 'low', 'close', 'volume', 'quote_asset_volume']:
                        df[col] = pd.to_numeric(df[col], errors='coerce')
                    
                    # Set datetime index
                    df.set_index('open_time', inplace=True)
                    
                    # Select only OHLCV columns needed for confluence
                    # Note: Using quote_asset_volume as it's the volume in BTC
                    result_df = df[['open', 'high', 'low', 'close', 'quote_asset_volume']].copy()
                    result_df.rename(columns={'quote_asset_volume': 'volume'}, inplace=True)
                    
                    return result_df
                    
                elif response.status == 429:
                    # Rate limit hit, wait a bit
                    await asyncio.sleep(1)
                    return None
                else:
                    return None
                    
        except Exception as e:
            logging.error(f"Error fetching klines for {symbol}: {str(e)}")
            return None

    async def send_telegram_alert(self, results):
        if not results:
            return
            
        try:
            message = f"🚨 Confluence Detection - Binance BTC Pairs {self.timeframe}\n\n"
            
            # Map timeframe to TradingView format - Extended mapping
            tv_timeframe_map = {
                # Minutes
                "1m": "1", "3m": "3", "5m": "5", "15m": "15", "30m": "30",
                # Hours  
                "1h": "60", "2h": "120", "4h": "240", "6h": "360", "8h": "480", "12h": "720",
                # Days
                "1d": "1D", "2d": "2D", "3d": "3D",
                # Weeks/Months
                "1w": "1W", "1M": "1M"
            }
            tv_timeframe = tv_timeframe_map.get(self.timeframe.lower(), self.timeframe)
            
            for result in results:
                formatted_symbol = result['symbol']
                tv_link = f"https://www.tradingview.com/chart/?symbol=BINANCE:{formatted_symbol}&interval={tv_timeframe}"
                
                # Escape HTML entities in the URL
                escaped_link = html.escape(tv_link)
                
                # Format according to BTC specifications
                time_str = ""
                if result.get('timestamp') is not None:
                    time_str = f"Time: {result['timestamp'].strftime('%Y-%m-%d %H:%M:%S')}\n"
                
                message += (
                    f"Symbol: {result['symbol']}\n"
                    f"{time_str}"
                    f"Volume BTC: ₿{result['volume_btc']:,.4f}\n"
                    f"Close: <a href='{escaped_link}'>₿{result['close']:.8f}</a>\n"
                    f"Volume Ratio: {result['volume_ratio']:.2f}x\n"
                    f"Close Off Low: {result['close_off_low']:.1f}%\n"
                    f"Momentum: {result['momentum_score']:.4f}\n"
                    f"{'='*30}\n"
                )
            
            # Split message more carefully to avoid breaking HTML tags
            max_length = 4000
            
            if len(message) > max_length:
                # Split at natural breaks (between results) to avoid breaking HTML
                sections = message.split('='*30 + '\n')
                current_chunk = ""
                
                for section in sections:
                    if len(current_chunk + section + '='*30 + '\n') > max_length:
                        if current_chunk:
                            await self.telegram_app.bot.send_message(
                                chat_id=self.telegram_chat_id,
                                text=current_chunk.strip(),
                                parse_mode='HTML',
                                disable_web_page_preview=True
                            )
                        current_chunk = section + '\n'
                    else:
                        current_chunk += section + '='*30 + '\n'
                
                # Send remaining chunk
                if current_chunk.strip():
                    await self.telegram_app.bot.send_message(
                        chat_id=self.telegram_chat_id,
                        text=current_chunk.strip(),
                        parse_mode='HTML',
                        disable_web_page_preview=True
                    )
            else:
                await self.telegram_app.bot.send_message(
                    chat_id=self.telegram_chat_id,
                    text=message,
                    parse_mode='HTML',
                    disable_web_page_preview=True
                )
                
        except Exception as e:
            logging.error(f"Error sending Telegram alert: {str(e)}")
            
            # Fallback: send without HTML formatting
            try:
                simple_message = f"🚨 Confluence Detection - Binance BTC Pairs {self.timeframe}\n\n"
                for result in results:
                    simple_message += (
                        f"Symbol: {result['symbol']}\n"
                        f"Volume BTC: ₿{result['volume_btc']:,.4f}\n"
                        f"Close: ₿{result['close']:.8f}\n"
                        f"Volume Ratio: {result['volume_ratio']:.2f}x\n"
                        f"Components: Vol={result['high_volume']}, Spread={result['spread_breakout']}, Mom={result['momentum_breakout']}\n\n"
                    )
                
                await self.telegram_app.bot.send_message(
                    chat_id=self.telegram_chat_id,
                    text=simple_message,
                    disable_web_page_preview=True
                )
            except Exception as fallback_error:
                logging.error(f"Fallback Telegram send also failed: {str(fallback_error)}")

    def scan_single_market(self, pair, df):
        """Scan a single market for Confluence pattern in the specified bar"""
        try:
            if df is None or len(df) < 50:  # Need enough data for confluence
                return None
            
            # Calculate which bar to check based on offset
            check_bar = -(self.offset + 1)  # offset=0 means current bar (-1), offset=1 means last closed (-2), etc.
            
            # Run confluence detection
            detected, result = detect_confluence(df, check_bar=check_bar)
            
            if detected:
                # Get the target bar values
                target_close = df['close'].iloc[check_bar]
                target_volume = df['volume'].iloc[check_bar]  # This is already in BTC
                
                confluence_result = {
                    'symbol': pair['symbol'],
                    'volume_btc': float(target_volume),
                    'close': float(target_close),
                    'volume': float(target_volume),
                    'volume_ratio': result['volume_ratio'],
                    'close_off_low': result['close_off_low'],
                    'momentum_score': result['momentum_score'],
                    'high_volume': result['high_volume'],
                    'spread_breakout': result['spread_breakout'],
                    'momentum_breakout': result['momentum_breakout'],
                    'bar_range': result['bar_range'],
                    'timestamp': df.index[check_bar] if hasattr(df.index, '__getitem__') else None
                }
                return confluence_result
                
        except Exception as e:
            logging.error(f"Error processing {pair['symbol']}: {str(e)}")
        return None

    async def scan_all_markets(self):
        """Scan all BTC markets for Confluence pattern"""
        await self.init_telegram()
        
        try:
            # Define volume thresholds for BTC pairs based on timeframe
            volume_thresholds = {
                # Minutes - higher volume needed for shorter timeframes
                "1m": 0.01,   "3m": 0.02,   "5m": 0.03,
                "15m": 0.05,  "30m": 0.08,
                # Hours
                "1h": 0.1,    "2h": 0.15,   "4h": 0.2,
                "6h": 0.25,   "8h": 0.3,    "12h": 0.35,
                # Days
                "1d": 0.4,    "2d": 0.8,    "3d": 1.2,
                # Weeks/Months
                "1w": 2.0,    "1M": 8.0
            }
            min_volume = volume_thresholds.get(self.timeframe.lower(), 0.1)  # Default 0.1 BTC
            
            # Create offset description
            if self.offset == 0:
                offset_desc = "current candle"
            elif self.offset == 1:
                offset_desc = "last closed candle"
            else:
                offset_desc = f"{self.offset} candles ago"
            
            print(f"Scanning Binance BTC pairs for Confluence patterns in {offset_desc}...")
            print(f"Timeframe: {self.timeframe}")
            print(f"Minimum volume threshold: ₿{min_volume:.2f}")
            
            # Get all BTC pairs from Binance
            print("Fetching BTC pairs from Binance...")
            btc_pairs = await self.get_btc_pairs()
            
            if not btc_pairs:
                print("No BTC pairs found or error fetching pairs")
                return []
            
            print(f"Found {len(btc_pairs)} BTC pairs to scan")
            
            # Filter out stablecoins and obvious non-trading pairs
            filtered_pairs = []
            skip_tokens = ['USDT', 'USDC', 'BUSD', 'DAI', 'TUSD', 'USDD', 'FDUSD']
            
            for pair in btc_pairs:
                if pair['baseAsset'] not in skip_tokens:
                    filtered_pairs.append(pair)
            
            print(f"After filtering: {len(filtered_pairs)} pairs to scan")
            
            # Get Binance timeframe - fallback to 1d if not found
            binance_interval = self.timeframe_map.get(self.timeframe.lower(), "1d")
            
            # Process all pairs with progress bar
            all_results = []
            successful_scans = 0
            
            with tqdm(total=len(filtered_pairs), desc="Scanning markets") as pbar:
                for pair in filtered_pairs:
                    try:
                        # Get OHLCV data from Binance
                        df = await self.get_klines(pair['symbol'], binance_interval, 100)
                        
                        if df is None or len(df) < 50:
                            pbar.update(1)
                            continue
                        
                        successful_scans += 1
                        target_idx = -(self.offset + 1)
                        
                        # Update progress bar with current symbol
                        pbar.set_description(f"Scanning: {pair['symbol']} ({len(df)} candles)")
                        
                        # Check volume threshold for the target candle
                        try:
                            target_candle_volume = float(df['volume'].iloc[target_idx])  # Already in BTC
                            
                            # Only process if volume meets threshold
                            if target_candle_volume >= min_volume:
                                result = self.scan_single_market(pair, df)
                                if result:
                                    all_results.append(result)
                                    print(f"Found Confluence: {pair['symbol']} 🎯")
                        except (IndexError, ValueError):
                            pass  # Skip if we can't calculate volume
                        
                        # Add small delay to respect rate limits
                        await asyncio.sleep(0.1)
                                
                    except Exception as e:
                        if "429" in str(e):
                            # Rate limit - add longer delay
                            await asyncio.sleep(2)
                    finally:
                        pbar.update(1)
            
            print(f"Successfully scanned {successful_scans}/{len(filtered_pairs)} pairs")
            
            # Sort by volume
            all_results.sort(key=lambda x: x['volume_btc'], reverse=True)
            
            # Send Telegram alert if we found any patterns
            if all_results:
                await self.send_telegram_alert(all_results)
            
            return all_results
            
        except Exception as e:
            logging.error(f"Error scanning markets: {str(e)}")
            return []
        finally:
            await self.close_session()

async def run_binance_btc_confluence_scanner(timeframe, offset=1):
    """
    Run the Binance BTC Confluence scanner
    
    Parameters:
    timeframe (str): Time period - Available options:
                    Minutes: 1m, 3m, 5m, 15m, 30m
                    Hours: 1h, 2h, 4h, 6h, 8h, 12h  
                    Days: 1d, 2d, 3d
                    Weeks/Months: 1w, 1M
    offset (int): Bar offset (0=current, 1=last closed, 2=two bars ago, etc.)
    """
    
    if offset == 0:
        offset_desc = "current candle"
    elif offset == 1:
        offset_desc = "last closed candle"
    else:
        offset_desc = f"{offset} candles ago"
    
    print(f"Starting Binance BTC Confluence scan for {offset_desc} on {timeframe}...")
    
    # Use the confluence telegram token
    telegram_token = "8066329517:AAHVr6kufZWe8UqCKPfmsRhSPleNlt_7G-g"
    telegram_chat_id = "375812423"
    
    scanner = BinanceBTCConfluenceScanner(telegram_token, telegram_chat_id, timeframe, offset)
    results = await scanner.scan_all_markets()
    
    if results:
        print(f"\nFound {len(results)} Confluence patterns:")
        
        # Convert results to DataFrame for console display
        df_results = pd.DataFrame(results)
        
        # Round numeric columns for BTC precision
        df_results['volume_btc'] = df_results['volume_btc'].round(4)
        df_results['close'] = df_results['close'].round(8)
        df_results['volume'] = df_results['volume'].round(4)
        df_results['volume_ratio'] = df_results['volume_ratio'].round(2)
        df_results['close_off_low'] = df_results['close_off_low'].round(1)
        df_results['momentum_score'] = df_results['momentum_score'].round(4)
        
        # Reorder columns for better display
        display_cols = ['symbol', 'close', 'volume_btc', 'volume_ratio', 'close_off_low', 
                       'momentum_score', 'high_volume', 'spread_breakout', 'momentum_breakout']
        available_cols = [col for col in display_cols if col in df_results.columns]
        
        # Display the results
        print(df_results[available_cols])
        
        # Show component analysis
        print(f"\n🔧 COMPONENT ANALYSIS:")
        vol_count = df_results['high_volume'].sum()
        spread_count = df_results['spread_breakout'].sum()
        momentum_count = df_results['momentum_breakout'].sum()
        
        print(f"High Volume signals: {vol_count}/{len(results)} ({vol_count/len(results)*100:.1f}%)")
        print(f"Spread Breakout signals: {spread_count}/{len(results)} ({spread_count/len(results)*100:.1f}%)")
        print(f"Momentum Breakout signals: {momentum_count}/{len(results)} ({momentum_count/len(results)*100:.1f}%)")
        
    else:
        print(f"\nNo Confluence patterns found in {offset_desc}")

# Set up logging
logging.basicConfig(level=logging.ERROR)

# Apply nest_asyncio to allow async operations in Jupyter
nest_asyncio.apply()

# Example usage functions
async def scan_binance_btc_current():
    """Scan current candle for confluence - Binance BTC pairs"""
    await run_binance_btc_confluence_scanner("1w", offset=0)

async def scan_binance_btc_closed():
    """Scan last closed candle for confluence - Binance BTC pairs"""
    await run_binance_btc_confluence_scanner("1w", offset=1)

async def scan_binance_btc_previous():
    """Scan two candles ago for confluence - Binance BTC pairs"""
    await run_binance_btc_confluence_scanner("1w", offset=2)

# Main execution function
async def main():
    """
    Main execution - modify parameters here
    """
    timeframe = "1w"  # 1d, 2d, 3d, 1w
    offset = 0        # 0 = current candle, 1 = last closed candle, 2 = two candles ago
    
    await run_binance_btc_confluence_scanner(timeframe, offset)

# Run the async main function
print("🔍 BINANCE BTC CONFLUENCE SCANNER")
print("=" * 40)
print("Available timeframes:")
print("• Minutes: 1m, 3m, 5m, 15m, 30m")
print("• Hours: 1h, 2h, 4h, 6h, 8h, 12h") 
print("• Days: 1d, 2d, 3d")
print("• Weeks/Months: 1w, 1M")
print("\nAvailable functions:")
print("• await main() - Run with default settings")
print("• await scan_binance_btc_current() - Scan current candle")
print("• await scan_binance_btc_closed() - Scan last closed candle")
print("• await scan_binance_btc_previous() - Scan two candles ago")
print("• await run_binance_btc_confluence_scanner('timeframe', offset) - Custom scan")
print("\nExamples:")
print("• await run_binance_btc_confluence_scanner('4h', 1)  # 4-hour last closed")
print("• await run_binance_btc_confluence_scanner('15m', 0) # 15-min current")
print("• await run_binance_btc_confluence_scanner('1M', 1)  # Monthly last closed")
print("This scanner uses REAL Binance BTC pair volumes!")

# Uncomment to auto-run:
await main()

In [ ]:
#Direct strategy debug of any pair on any exchange
import asyncio
import sys
import os
import logging
import pandas as pd
import numpy as np
logging.basicConfig(level=logging.DEBUG, format='%(levelname)s: %(message)s')
project_dir = os.path.join(os.getcwd(), "Project")
sys.path.insert(0, project_dir)
print(f"✓ Added {project_dir} to sys.path")
from exchanges import MexcSpotClient, BybitSpotClient, GateioSpotClient, KucoinSpotClient, BinanceSpotClient, BinanceFuturesClient, BybitFuturesClient
from custom_strategies import detect_volume_surge, detect_weak_uptrend, detect_pin_down
from breakout_vsa import vsa_detector, breakout_bar_vsa, stop_bar_vsa, reversal_bar_vsa, start_bar_vsa, loaded_bar_vsa, test_bar_vsa

async def test_strategy(exchange_client_class, timeframe, symbol, strategy_name):
    client = exchange_client_class(timeframe=timeframe)
    await client.init_session()
    df = await client.fetch_klines(symbol)
    await client.close_session()
    
    if df is None or len(df) < 10:
        print(f"No data fetched for {symbol} or insufficient data (< 10 bars)")
        return
    
    print(f"{timeframe} Candles for {symbol}:")
    print(df.tail(5))
    last_row = df.iloc[-1]
    volume_usd = last_row['volume'] * last_row['close']
    print(f"Last Bar: volume_usd={volume_usd:.2f}, close={last_row['close']}, volume={last_row['volume']:.2f}")
    
    # Different handling based on strategy type
    if strategy_name == "volume_surge":
        # Use detect_volume_surge directly
        detected, result = detect_volume_surge(df)
        
        print(f"\nVolume Surge Detection Results:")
        print(f"Detected: {detected}")
        
        if detected:
            print(f"\nVolume Surge Details:")
            print(f"  Date: {result['timestamp']}")
            print(f"  Close: ${result['close_price']:,.8f}")
            print(f"  Volume: {result['volume']:,.2f}")
            print(f"  Volume USD: ${result['volume_usd']:,.2f}")
            print(f"  Volume Ratio: {result['volume_ratio']:,.2f}x")
            print(f"  Score: {result['score']:,.2f}")
            print(f"  Price Extreme: {result['price_extreme']}")
    
    elif strategy_name == "pin_down":
        from custom_strategies import detect_pin_down
        detected, result = detect_pin_down(df)
        
        print(f"\nPin Down Detection Results:")
        print(f"Detected: {detected}")
        
        if detected:
            print(f"\nPin Down Details:")
            for key, value in result.items():
                if key != 'symbol':  # Skip symbol as we already know it
                    print(f"  {key}: {value}")
    
    elif strategy_name == "weak_uptrend":
        from custom_strategies import detect_weak_uptrend
        detected, result = detect_weak_uptrend(df)
        
        print(f"\nWeak Uptrend Detection Results:")
        print(f"Detected: {detected}")
        
        if detected:
            print(f"\nWeak Uptrend Details:")
            for key, value in result.items():
                if key != 'symbol':  # Skip symbol as we already know it
                    print(f"  {key}: {value}")
    
    else:
        # For VSA strategies, import the appropriate get_params
        if strategy_name == "reversal_bar":
            from breakout_vsa.strategies.reversal_bar import get_params
        elif strategy_name == "breakout_bar":
            from breakout_vsa.strategies.breakout_bar import get_params
        elif strategy_name == "loaded_bar":
            from breakout_vsa.strategies.loaded_bar import get_params
        elif strategy_name == "stop_bar":
            from breakout_vsa.strategies.stop_bar import get_params
        elif strategy_name == "start_bar":
            from breakout_vsa.strategies.start_bar import get_params
        else:
            print(f"Unknown strategy: {strategy_name}")
            return
        
        # Use vsa_detector with strategy-specific params
        params = get_params()
        condition, result = vsa_detector(df, params)
        
        strategy_display_name = strategy_name.replace('_vsa', '').replace('_', ' ').title()
        print(f"\n{strategy_display_name} Detection Results:")
        print(f"Current Bar (index -1): {condition.iloc[-1]}")
        if len(df) > 1:
            print(f"Last Closed Bar (index -2): {condition.iloc[-2]}")
        
        if condition.iloc[-1] or (len(df) > 1 and condition.iloc[-2]):
            detected_idx = -1 if condition.iloc[-1] else -2
            volume_mean = df['volume'].rolling(7).mean().iloc[detected_idx]
            bar_range = df['high'].iloc[detected_idx] - df['low'].iloc[detected_idx]
            close_off_low = (df['close'].iloc[detected_idx] - df['low'].iloc[detected_idx]) / bar_range * 100 if bar_range > 0 else 0
            volume_usd_detected = df['volume'].iloc[detected_idx] * df['close'].iloc[detected_idx]
            
            arctan_ratio = result['arctan_ratio'].iloc[detected_idx]  # From result DataFrame
            
            print(f"\nDetected at index {detected_idx} ({'Current' if detected_idx == -1 else 'Last Closed'} Bar):")
            print(f"  Date: {df.index[detected_idx]}")
            print(f"  Close: ${df['close'].iloc[detected_idx]:,.8f}")
            print(f"  Volume Ratio: {df['volume'].iloc[detected_idx] / volume_mean if volume_mean > 0 else 0:.2f}x")
            print(f"  {timeframe} Volume: ${volume_usd_detected:.2f}")
            print(f"  Close Off Low: {close_off_low:.1f}%")
            print(f"  Angular Ratio: {arctan_ratio:.2f}")

# Define the test case
exchange_client = GateioSpotClient
timeframe = "1w"
symbol = "PRCL_USDT"
strategy = "loaded_bar"
await test_strategy(exchange_client, timeframe, symbol, strategy)

In [ ]:
#zip the project

import shutil
import os

# Go to parent directory of your project
os.chdir("/home/jovyan/work/Crypto/sevenfigures-bot/hbs_2025")

# Create the zip file (this will include everything inside 'hbs_2025')
shutil.make_archive("Project_VSA_2025_backup", 'zip', "Project")


In [ ]:
#!/usr/bin/env python3 -> Debug built weekly candles for mexc and kucoin 
import sys
import os
project_dir = os.path.join(os.getcwd(), "Project")
sys.path.insert(0, project_dir)
print(f"✓ Added {project_dir} to sys.path")
import asyncio
import logging
import pandas as pd
import numpy as np
from exchanges.kucoin_client import KucoinClient
from breakout_vsa.core import calculate_start_bar

from scanner.main import kline_cache
kline_cache.clear()  # Clear cache for fresh data

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s: %(message)s')

async def debug_start_bar_detection():
    # Initialize client
    client = KucoinClient(timeframe="1w")
    await client.init_session()
    
    # Symbol to debug
    symbol = "TAO-USDT"
    
    try:
        # Fetch data
        df = await client.fetch_klines(symbol)
        
        if df is not None:
            print(f"Weekly candles for {symbol}:")
            print(df.tail())
            
            # Add intermediate calculations to see what's happening
            # This is a modified version of calculate_start_bar that adds debugging
            lookback = 5
            volume_lookback = 30
            volume_percentile = 50
            low_percentile = 75
            range_percentile = 75
            close_off_lows_percent = 50
            prev_close_range = 75
            
            # Calculate basic bar characteristics
            df['bar_range'] = df['high'] - df['low']
            df['volume_rank'] = df['volume'].rolling(lookback).apply(
                lambda x: sum(1.0 for val in x if val <= x[-1]) / len(x) * 100, 
                raw=True
            )
            
            # Calculate rolling values
            df['macro_low'] = df['low'].rolling(volume_lookback).min()
            df['macro_high'] = df['high'].rolling(volume_lookback).max()
            df['highest_high'] = df['high'].rolling(lookback).max()
            
            # Volume conditions
            df['volume_sma'] = df['volume'].rolling(volume_lookback).mean()
            df['volume_std'] = df['volume'].rolling(volume_lookback).std()
            df['excess_volume'] = df['volume'] > (df['volume_sma'] + 3.0 * df['volume_std'])
            
            # Range conditions
            df['range_sma'] = df['bar_range'].rolling(volume_lookback).mean()
            df['range_std'] = df['bar_range'].rolling(volume_lookback).std()
            df['excess_range'] = df['bar_range'] > (df['range_sma'] + 3.0 * df['range_std'])
            
            # Volume percentile condition
            def is_in_top_percent(series, length, percent):
                ranks = series.rolling(length).apply(
                    lambda x: sum(1.0 for val in x if val <= x[-1]) / len(x) * 100, 
                    raw=True
                )
                return ranks >= percent
            
            def is_in_bottom_percent(series, length, percent):
                ranks = series.rolling(length).apply(
                    lambda x: sum(1.0 for val in x if val <= x[-1]) / len(x) * 100, 
                    raw=True
                )
                return ranks <= percent
            
            # Volume conditions
            df['is_higher_volume'] = is_in_top_percent(df['volume'], lookback, volume_percentile)
            df['is_high_volume'] = (df['volume'] > 0.75 * df['volume_sma']) & (df['volume'] > df['volume'].shift(1))
            
            # Price action conditions
            df['has_higher_high'] = df['high'] > df['high'].shift(1)
            df['no_narrow_range'] = is_in_top_percent(df['bar_range'], lookback, range_percentile)
            
            # Low price condition
            df['is_in_the_lows'] = (
                (df['low'] - df['macro_low']).abs() < df['bar_range']
            ) | is_in_bottom_percent(df['low'], volume_lookback, low_percentile)
            
            # Close position conditions
            df['close_in_the_highs'] = (
                (df['close'] - df['low']) / df['bar_range']
            ) >= (close_off_lows_percent / 100)
            
            # Previous close distance condition
            df['far_prev_close'] = (
                (df['close'] - df['close'].shift(1)).abs() >=
                (df['bar_range'].shift(1) * (prev_close_range / 100))
            )
            
            # New highs condition
            df['new_highs'] = df['high'] >= 0.75 * df['highest_high']
            
            # Optional strength condition
            df['strong_close'] = df['close'] >= df['highest_high'].shift(1)
            
            # Now check the actual values for the last few bars
            last_rows = df.tail(3)
            
            print("\nAnalyzing last 3 bars:")
            for idx, row in last_rows.iterrows():
                print(f"\nBar at {idx.strftime('%Y-%m-%d')}:")
                print(f"  is_high_volume: {row['is_high_volume']}")
                print(f"  has_higher_high: {row['has_higher_high']}")
                print(f"  no_narrow_range: {row['no_narrow_range']}")
                print(f"  close_in_the_highs: {row['close_in_the_highs']}")
                print(f"  far_prev_close: {row['far_prev_close']}")
                print(f"  excess_range: {row['excess_range']}")
                print(f"  excess_volume: {row['excess_volume']}")
                print(f"  new_highs: {row['new_highs']}")
                print(f"  is_in_the_lows: {row['is_in_the_lows']}")
                print(f"  volume: {row['volume']}, volume_sma: {row['volume_sma']}")
                print(f"  bar_range: {row['bar_range']}, range_sma: {row['range_sma']}")
                
            # Run the original function to confirm
            start_bar_pattern = calculate_start_bar(df)
            print(f"\nFinal Start Bar detection result:")
            print(start_bar_pattern.tail(3))
            
    except Exception as e:
        print(f"Error in debug: {str(e)}")
    finally:
        await client.close_session()

# Replace the last part of your script with this:
if __name__ == "__main__":
    try:
        # For Jupyter/IPython environments
        import nest_asyncio
        nest_asyncio.apply()
        asyncio.run(debug_start_bar_detection())
    except ImportError:
        # For regular Python environments
        asyncio.run(debug_start_bar_detection())

In [3]:
#zip the project

import shutil
import os

# Go to parent directory of your project
os.chdir("/home/jovyan/work/Crypto/sevenfigures-bot/hbs_2025")

# Create the zip file (this will include everything inside 'hbs_2025')
shutil.make_archive("Project_VSA_2025_backup", 'zip', "Project")

'/home/jovyan/work/Crypto/sevenfigures-bot/hbs_2025/Project_VSA_2025_backup.zip'

In [ ]:
# ARCHIVE - Confluence Scanner with bar offset

from telegram.ext import Application
import logging
import pandas as pd
import numpy as np
import asyncio
import logging
import nest_asyncio
from datetime import datetime
from tqdm.asyncio import tqdm
import sys
import os
import html

# Add project path
project_dir = os.path.join(os.getcwd(), "Project")
sys.path.insert(0, project_dir)

from exchanges.sf_pairs_service import SFPairsService
from custom_strategies import detect_confluence

class ConfluenceScanner:
    def __init__(self, telegram_token, telegram_chat_id, exchange, timeframe, offset=1):
        self.telegram_token = telegram_token
        self.telegram_chat_id = telegram_chat_id
        self.telegram_app = None
        self.exchange = exchange
        self.timeframe = timeframe
        self.offset = offset  # Added offset parameter
        self.sf_service = SFPairsService()
        
    async def init_telegram(self):
        if self.telegram_app is None:
            self.telegram_app = Application.builder().token(self.telegram_token).build()

    async def send_telegram_alert(self, results):
        if not results:
            return
            
        try:
            message = f"🚨 Confluence Detection - {self.exchange} {self.timeframe}\n\n"
            
            # Map timeframe to TradingView format
            tv_timeframe_map = {
                "1d": "1D",
                "2d": "2D",
                "1w": "1W"
            }
            tv_timeframe = tv_timeframe_map.get(self.timeframe.lower(), self.timeframe)
            
            for result in results:
                exchange_name = self.exchange.upper()
                formatted_symbol = f"{result['symbol']}"
                tv_link = f"https://www.tradingview.com/chart/?symbol={exchange_name}:{formatted_symbol}&interval={tv_timeframe}"
                
                # Escape HTML entities in the URL
                escaped_link = html.escape(tv_link)
                
                # Format according to specified requirements
                time_str = ""
                if result.get('timestamp') is not None:
                    time_str = f"Time: {result['timestamp'].strftime('%Y-%m-%d %H:%M:%S')}\n"
                
                message += (
                    f"Symbol: {result['symbol']}\n"
                    f"{time_str}"
                    f"Volume USD: ${result['volume_usd']:,.2f}\n"
                    f"Close: <a href='{escaped_link}'>${result['close']:,.8f}</a>\n"
                    f"Volume Ratio: {result['volume_ratio']:.2f}x\n"
                    f"Close Off Low: {result['close_off_low']:.1f}%\n"
                    f"{'='*30}\n"
                )
            
            # Split message more carefully to avoid breaking HTML tags
            max_length = 4000  # Reduced from 4096 to be safer
            
            if len(message) > max_length:
                # Split at natural breaks (between results) to avoid breaking HTML
                sections = message.split('='*30 + '\n')
                current_chunk = ""
                
                for section in sections:
                    if len(current_chunk + section + '='*30 + '\n') > max_length:
                        if current_chunk:
                            await self.telegram_app.bot.send_message(
                                chat_id=self.telegram_chat_id,
                                text=current_chunk.strip(),
                                parse_mode='HTML',
                                disable_web_page_preview=True
                            )
                        current_chunk = section + '\n'
                    else:
                        current_chunk += section + '='*30 + '\n'
                
                # Send remaining chunk
                if current_chunk.strip():
                    await self.telegram_app.bot.send_message(
                        chat_id=self.telegram_chat_id,
                        text=current_chunk.strip(),
                        parse_mode='HTML',
                        disable_web_page_preview=True
                    )
            else:
                await self.telegram_app.bot.send_message(
                    chat_id=self.telegram_chat_id,
                    text=message,
                    parse_mode='HTML',
                    disable_web_page_preview=True
                )
                
        except Exception as e:
            logging.error(f"Error sending Telegram alert: {str(e)}")
            
            # Fallback: send without HTML formatting
            try:
                simple_message = f"🚨 Confluence Detection - {self.exchange} {self.timeframe}\n\n"
                for result in results:
                    simple_message += (
                        f"Symbol: {result['symbol']}\n"
                        f"Volume USD: ${result['volume_usd']:,.2f}\n"
                        f"Close: ${result['close']:,.8f}\n"
                        f"Volume Ratio: {result['volume_ratio']:.2f}x\n"
                        f"Components: Vol={result['high_volume']}, Spread={result['spread_breakout']}, Mom={result['momentum_breakout']}\n\n"
                    )
                
                await self.telegram_app.bot.send_message(
                    chat_id=self.telegram_chat_id,
                    text=simple_message,
                    disable_web_page_preview=True
                )
            except Exception as fallback_error:
                logging.error(f"Fallback Telegram send also failed: {str(fallback_error)}")

    def prepare_sf_data(self, raw_df):
        """Convert SF data to confluence-compatible format"""
        if raw_df is None or len(raw_df) == 0:
            return None
        
        df = pd.DataFrame(raw_df)
        
        # Convert datetime column to pandas datetime and set as index
        if 'datetime' in df.columns:
            df['datetime'] = pd.to_datetime(df['datetime'])
            df = df.set_index('datetime')
        elif 'time' in df.columns:
            # Convert Unix timestamp to datetime
            df['time'] = pd.to_datetime(df['time'], unit='ms')
            df = df.set_index('time')
        
        # Select only OHLCV columns needed for confluence
        required_cols = ['open', 'high', 'low', 'close', 'volume']
        available_cols = [col for col in required_cols if col in df.columns]
        
        if len(available_cols) != 5:
            return None
        
        # Select and clean data
        result_df = df[required_cols].copy()
        
        # Ensure numeric types
        for col in required_cols:
            result_df[col] = pd.to_numeric(result_df[col], errors='coerce')
        
        # Drop any NaN rows
        result_df = result_df.dropna()
        
        return result_df

    def scan_single_market(self, pair, ohlcv_data):
        """Scan a single market for Confluence pattern in the specified bar"""
        try:
            # Prepare data for confluence analysis
            df = self.prepare_sf_data(ohlcv_data)
            
            if df is None or len(df) < 50:  # Need enough data for confluence
                return None
            
            # Calculate which bar to check based on offset
            check_bar = -(self.offset + 1)  # offset=0 means current bar (-1), offset=1 means last closed (-2), etc.
            
            # Run confluence detection
            detected, result = detect_confluence(df, check_bar=check_bar)
            
            if detected:
                # Calculate volume in USD for the target bar
                target_close = df['close'].iloc[check_bar]
                target_volume = df['volume'].iloc[check_bar]
                volume_usd = float(target_close) * float(target_volume)
                
                confluence_result = {
                    'symbol': f"{pair['Token']}{pair['Quote']}",
                    'volume_usd': volume_usd,
                    'close': float(target_close),
                    'volume': float(target_volume),
                    'volume_ratio': result['volume_ratio'],
                    'close_off_low': result['close_off_low'],
                    'momentum_score': result['momentum_score'],
                    'high_volume': result['high_volume'],
                    'spread_breakout': result['spread_breakout'],
                    'momentum_breakout': result['momentum_breakout'],
                    'bar_range': result['bar_range']
                }
                return confluence_result
                
        except Exception as e:
            logging.error(f"Error processing {pair['Token']}{pair['Quote']}: {str(e)}")
        return None

    async def scan_all_markets(self):
        """Scan all markets for Confluence pattern"""
        await self.init_telegram()
        try:
            # Define volume thresholds
            volume_thresholds = {
                "1w": 300000,
                "2d": 100000,
                "1d": 50000
            }
            min_volume = volume_thresholds.get(self.timeframe.lower(), 50000)
            
            # Create offset description
            if self.offset == 0:
                offset_desc = "current candle"
            elif self.offset == 1:
                offset_desc = "last closed candle"
            else:
                offset_desc = f"{self.offset} candles ago"
            
            print(f"Scanning for Confluence patterns in {offset_desc}...")
            print(f"Minimum volume threshold: ${min_volume:,.0f}")
            
            # Get all pairs from SF service
            pairs = self.sf_service.get_pairs_of_exchange(self.exchange)
            print(f"Found {len(pairs)} markets to scan...")
            
            # Process all pairs with progress bar
            all_results = []
            with tqdm(total=len(pairs), desc="Scanning markets") as pbar:
                for pair in pairs:
                    try:
                        # Get OHLCV data from SF service
                        ohlcv_data = self.sf_service.get_ohlcv_for_pair(
                            pair['Token'], 
                            pair['Quote'], 
                            self.exchange, 
                            self.timeframe, 
                            100  # Get more data for confluence analysis
                        )
                        
                        if ohlcv_data is None or len(ohlcv_data) == 0:
                            pbar.update(1)
                            continue
                        
                        df = pd.DataFrame(ohlcv_data)
                        
                        # Check if we have enough data
                        if len(df) >= 50:  # Need enough for confluence analysis
                            target_idx = -(self.offset + 1)  # Adjust index based on offset
                            
                            # Check volume threshold for the target candle
                            try:
                                target_candle_volume = float(df['close'].iloc[target_idx]) * float(df['volume'].iloc[target_idx])
                                
                                # Only process if volume meets threshold
                                if target_candle_volume >= min_volume:
                                    result = self.scan_single_market(pair, ohlcv_data)
                                    if result:
                                        all_results.append(result)
                                        print(f"Found Confluence: {pair['Token']}{pair['Quote']} 🎯")
                            except (IndexError, ValueError):
                                pass  # Skip if we can't calculate volume
                                    
                    except Exception as e:
                        if "500" not in str(e):  # Don't log 500 errors
                            logging.error(f"Error processing {pair['Token']}{pair['Quote']}: {str(e)}")
                    finally:
                        pbar.update(1)
            
            # Sort by volume
            all_results.sort(key=lambda x: x['volume_usd'], reverse=True)
            
            # Send Telegram alert if we found any patterns
            if all_results:
                await self.send_telegram_alert(all_results)
            
            return all_results
            
        except Exception as e:
            logging.error(f"Error scanning markets: {str(e)}")
            return []

async def run_confluence_scanner(exchange, timeframe, offset=1):
    """
    Run the Confluence scanner
    
    Parameters:
    exchange (str): Exchange name (Kucoin, Mexc, Binance)
    timeframe (str): Time period (1d, 2d, 1w)
    offset (int): Bar offset (0=current, 1=last closed, 2=two bars ago, etc.)
    """
    
    if offset == 0:
        offset_desc = "current candle"
    elif offset == 1:
        offset_desc = "last closed candle"
    else:
        offset_desc = f"{offset} candles ago"
    
    print(f"Starting Confluence scan for {offset_desc} on {exchange} {timeframe}...")
    
    # Use the confluence telegram token from your big project config
    # You should replace this with the actual token from utils/config.py TELEGRAM_TOKENS["confluence"]
    telegram_token = "8066329517:AAHVr6kufZWe8UqCKPfmsRhSPleNlt_7G-g"  # Replace with confluence token
    telegram_chat_id = "375812423"  # Your chat ID
    
    scanner = ConfluenceScanner(telegram_token, telegram_chat_id, exchange, timeframe, offset)
    results = await scanner.scan_all_markets()
    
    if results:
        print(f"\nFound {len(results)} Confluence patterns:")
        
        # Convert results to DataFrame for console display
        df_results = pd.DataFrame(results)
        
        # Round numeric columns
        df_results['volume_usd'] = df_results['volume_usd'].round(2)
        df_results['close'] = df_results['close'].round(8)
        df_results['volume'] = df_results['volume'].round(2)
        df_results['volume_ratio'] = df_results['volume_ratio'].round(2)
        df_results['close_off_low'] = df_results['close_off_low'].round(1)
        df_results['momentum_score'] = df_results['momentum_score'].round(4)
        
        # Reorder columns for better display
        display_cols = ['symbol', 'close', 'volume_usd', 'volume_ratio', 'close_off_low', 
                       'momentum_score', 'high_volume', 'spread_breakout', 'momentum_breakout']
        available_cols = [col for col in display_cols if col in df_results.columns]
        
        # Display the results
        print(df_results[available_cols])
        
        # Show component analysis
        print(f"\n🔧 COMPONENT ANALYSIS:")
        vol_count = df_results['high_volume'].sum()
        spread_count = df_results['spread_breakout'].sum()
        momentum_count = df_results['momentum_breakout'].sum()
        
        print(f"High Volume signals: {vol_count}/{len(results)} ({vol_count/len(results)*100:.1f}%)")
        print(f"Spread Breakout signals: {spread_count}/{len(results)} ({spread_count/len(results)*100:.1f}%)")
        print(f"Momentum Breakout signals: {momentum_count}/{len(results)} ({momentum_count/len(results)*100:.1f}%)")
        
    else:
        print(f"\nNo Confluence patterns found in {offset_desc}")

# Set up logging
logging.basicConfig(level=logging.ERROR)

# Apply nest_asyncio to allow async operations in Jupyter
nest_asyncio.apply()

# Example usage functions
async def scan_current_confluence():
    """Scan current candle for confluence"""
    await run_confluence_scanner("Kucoin", "1w", offset=0)

async def scan_closed_confluence():
    """Scan last closed candle for confluence"""
    await run_confluence_scanner("Kucoin", "1w", offset=1)

async def scan_previous_confluence():
    """Scan two candles ago for confluence"""
    await run_confluence_scanner("Kucoin", "1w", offset=2)

# Main execution function
async def main():
    """
    Main execution - modify parameters here
    """
    exchange = "Mexc"  # Binance, Kucoin, Mexc
    timeframe = "1w"     # 1d, 2d, 1w
    offset = 0           # 0 = current candle, 1 = last closed candle, 2 = two candles ago
    
    await run_confluence_scanner(exchange, timeframe, offset)

# Run the async main function
print("🔍 CONFLUENCE SCANNER")
print("=" * 30)
print("Available functions:")
print("• await main() - Run with default settings")
print("• await scan_current_confluence() - Scan current candle")
print("• await scan_closed_confluence() - Scan last closed candle")
print("• await scan_previous_confluence() - Scan two candles ago")
print("• await run_confluence_scanner('Exchange', 'timeframe', offset) - Custom scan")
print("\nExample: await main()")

# Uncomment to auto-run:
await main()

In [ ]:
# Standalone HLC Bar Chart Plotter
"""
Standalone HLC Bar Chart Plotter
A reusable function for plotting HLC bars with optional pattern highlighting
"""

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
from matplotlib.lines import Line2D

def plot_hlc_bars(data, highlighted_bars=None, title="HLC Chart", symbol="SYMBOL", 
                  interval="1d", semilog=False, highlight_color="fuchsia", 
                  highlight_label="Pattern", figsize=(14, 10), show_volume=True):
    """
    Plot HLC bar chart with optional pattern highlighting
    
    Parameters:
    -----------
    data : pandas.DataFrame
        DataFrame with columns: ['datetime', 'high', 'low', 'close', 'volume']
        - datetime: timestamp column (will be used for x-axis)
        - high: high prices
        - low: low prices  
        - close: close prices
        - volume: volume data (optional if show_volume=False)
        
    highlighted_bars : pandas.Series or list/array, optional
        Boolean series or array indicating which bars to highlight
        Length must match data length
        
    title : str, default "HLC Chart"
        Chart title
        
    symbol : str, default "SYMBOL" 
        Symbol name for display
        
    interval : str, default "1d"
        Time interval for date formatting (1m, 5m, 15m, 30m, 1h, 4h, 1d, 3d, 1w, 1M)
        
    semilog : bool, default False
        Use logarithmic scale for price chart
        
    highlight_color : str, default "fuchsia"
        Color for highlighted bars
        
    highlight_label : str, default "Pattern"
        Label for highlighted bars in legend
        
    figsize : tuple, default (14, 10)
        Figure size (width, height)
        
    show_volume : bool, default True
        Whether to show volume subplot
        
    Returns:
    --------
    matplotlib.figure.Figure
        The created figure object
    """
    
    # Input validation
    if not isinstance(data, pd.DataFrame):
        raise ValueError("data must be a pandas DataFrame")
    
    # Check required columns
    required_cols = ['datetime', 'high', 'low', 'close']
    if show_volume:
        required_cols.append('volume')
    
    missing_cols = [col for col in required_cols if col not in data.columns]
    if missing_cols:
        raise ValueError(f"Missing required columns: {missing_cols}")
    
    if len(data) == 0:
        raise ValueError("data DataFrame is empty")
    
    # Validate highlighted_bars
    if highlighted_bars is not None:
        if len(highlighted_bars) != len(data):
            raise ValueError("highlighted_bars length must match data length")
        # Convert to boolean array
        highlighted_bars = np.array(highlighted_bars, dtype=bool)
    else:
        highlighted_bars = np.zeros(len(data), dtype=bool)
    
    # Create figure with subplots
    if show_volume:
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=figsize, 
                                       gridspec_kw={'height_ratios': [3, 1]})
    else:
        fig, ax1 = plt.subplots(1, 1, figsize=figsize)
        ax2 = None
    
    # Convert dates to numbers for plotting
    dates = mdates.date2num(data['datetime'])
    
    # Calculate margins and tick length
    date_range = dates[-1] - dates[0] if len(dates) > 1 else 1
    margin = date_range * 0.05
    
    # Calculate actual bar spacing for consistent tick length
    if len(dates) > 1:
        avg_bar_spacing = date_range / (len(dates) - 1)
        tick_length = avg_bar_spacing * 0.4  # 40% of bar spacing
        volume_bar_width = avg_bar_spacing * 0.8
    else:
        tick_length = date_range * 0.01
        volume_bar_width = date_range * 0.02
    
    # Draw HLC bars
    for i, (date, high, low, close) in enumerate(zip(dates, data['high'], data['low'], data['close'])):
        is_highlighted = highlighted_bars[i]
        color = highlight_color if is_highlighted else 'black'
        line_width = 1.2
        
        # Vertical line from low to high
        ax1.plot([date, date], [low, high], color=color, linewidth=line_width, solid_capstyle='butt')
        
        # Horizontal tick mark for close (on the right side)
        ax1.plot([date, date + tick_length], [close, close], color=color, 
                linewidth=line_width+0.5, solid_capstyle='butt')
    
    # Configure price chart
    if semilog:
        ax1.set_yscale('log')
        scale_text = "Semilog Scale"
    else:
        scale_text = "Linear Scale"
    
    ax1.set_title(f'{symbol} {title} - {scale_text}', fontsize=16, fontweight='bold')
    ax1.set_ylabel('Price', fontsize=12)
    ax1.grid(True, alpha=0.3)
    
    # Format x-axis based on timeframe
    _format_datetime_axis(ax1, interval)
    ax1.set_xlim(dates[0] - margin, dates[-1] + margin)
    
    # Volume chart
    if show_volume and ax2 is not None:
        volume_colors = [highlight_color if highlighted_bars[i] else 'orange' for i in range(len(dates))]
        volume_edges = ['darkmagenta' if highlighted_bars[i] else 'darkorange' for i in range(len(dates))]
        
        ax2.bar(dates, data['volume'], width=volume_bar_width, alpha=0.6, 
                color=volume_colors, edgecolor=volume_edges)
        ax2.set_ylabel('Volume', fontsize=12)
        ax2.set_xlabel('Date', fontsize=12)
        ax2.grid(True, alpha=0.3)
        
        _format_datetime_axis(ax2, interval)
        ax2.set_xlim(dates[0] - margin, dates[-1] + margin)
        plt.setp(ax2.xaxis.get_majorticklabels(), rotation=45)
    else:
        ax1.set_xlabel('Date', fontsize=12)
    
    # Create legend
    legend_elements = [
        Line2D([0], [0], color='black', linewidth=2, label='High-Low Range'),
        Line2D([0], [0], color='black', linewidth=3, label='Close Price (right tick)')
    ]
    
    # Add highlighted bars to legend if any exist
    if highlighted_bars.any():
        legend_elements.append(
            Line2D([0], [0], color=highlight_color, linewidth=3, label=highlight_label)
        )
        highlight_count = highlighted_bars.sum()
    else:
        highlight_count = 0
    
    ax1.legend(handles=legend_elements, loc='upper left', title=scale_text)
    
    # Add pattern count if patterns exist
    if highlight_count > 0:
        ax1.text(0.99, 0.95, f'{highlight_label}: {highlight_count}', 
                transform=ax1.transAxes, ha='right', va='top',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8),
                fontsize=10)
    
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)
    plt.tight_layout()
    
    return fig

def _format_datetime_axis(ax, interval):
    """Helper function to format datetime axis based on interval"""
    if interval in ['1m', '5m', '15m', '30m']:
        ax.xaxis.set_major_locator(mdates.HourLocator(interval=6))
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d %H:%M'))
    elif interval in ['1h', '2h', '4h', '6h', '12h']:
        ax.xaxis.set_major_locator(mdates.DayLocator(interval=1))
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    elif interval in ['1d', '3d']:
        ax.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
    else:  # weekly, monthly
        ax.xaxis.set_major_locator(mdates.MonthLocator(interval=6))
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))

# Example usage and test function
def example_usage():
    """Example showing how to use the plot_hlc_bars function"""
    
    # Create sample data
    import datetime
    dates = pd.date_range('2023-01-01', periods=100, freq='D')
    np.random.seed(42)
    
    # Generate realistic OHLC data
    closes = 100 + np.cumsum(np.random.randn(100) * 0.02)
    highs = closes + np.random.rand(100) * 5
    lows = closes - np.random.rand(100) * 5
    volumes = np.random.rand(100) * 1000000
    
    data = pd.DataFrame({
        'datetime': dates,
        'high': highs,
        'low': lows, 
        'close': closes,
        'volume': volumes
    })
    
    # Create some random pattern detections
    pattern_detected = np.random.choice([True, False], size=100, p=[0.1, 0.9])
    
    # Plot with pattern highlighting
    fig = plot_hlc_bars(
        data=data,
        highlighted_bars=pattern_detected,
        title="Daily Chart with Pattern Detection",
        symbol="EXAMPLE",
        interval="1d",
        semilog=False,
        highlight_color="red",
        highlight_label="Detected Pattern"
    )
    
    plt.show()
    return fig

# Scanner integration example
def scanner_integration_example():
    """Example of how to integrate with a scanner function"""
    
    def my_pattern_scanner(data):
        """
        Example scanner function - replace with your actual scanner logic
        Returns boolean array indicating pattern detection
        """
        # Example: detect when close > 20-period moving average
        ma20 = data['close'].rolling(20).mean()
        pattern = (data['close'] > ma20) & (data['volume'] > data['volume'].rolling(10).mean())
        return pattern.fillna(False)
    
    # Your data loading logic here
    # data = load_your_data()  # Replace with actual data loading
    
    # For demo, create sample data
    dates = pd.date_range('2023-01-01', periods=200, freq='D')
    np.random.seed(42)
    closes = 100 + np.cumsum(np.random.randn(200) * 0.02)
    highs = closes + np.random.rand(200) * 3
    lows = closes - np.random.rand(200) * 3
    volumes = np.random.rand(200) * 1000000
    
    data = pd.DataFrame({
        'datetime': dates,
        'high': highs,
        'low': lows,
        'close': closes,
        'volume': volumes
    })
    
    # Run your scanner
    detected_patterns = my_pattern_scanner(data)
    
    # Plot only if patterns are detected
    if detected_patterns.any():
        print(f"Patterns detected! Found {detected_patterns.sum()} occurrences")
        fig = plot_hlc_bars(
            data=data,
            highlighted_bars=detected_patterns,
            title="Scanner Results",
            symbol="SCANNED_SYMBOL",
            interval="1d",
            highlight_color="lime",
            highlight_label="Scanner Hit"
        )
        plt.show()
    else:
        print("No patterns detected")

if __name__ == "__main__":
    # Run example
    example_usage()

In [ ]:
#Dashboard
from pyngrok import ngrok
import os, sys, subprocess, time

# --- ngrok auth (DO NOT commit your token) ---
ngrok.set_auth_token("31mFDQNYuBuJw7mTKNxyDZLbZag_4Q1mV2EEggMBGYecRRZyF")

# Clean old tunnels
try:
    ngrok.kill()
except Exception:
    pass

# Pick a port
port = 8501

# Launch Streamlit dashboard in the background
# Ensure working directory contains dashboard.py
cmd = ["streamlit", "run", "dashboard.py", "--server.address", "0.0.0.0", "--server.port", str(port)]
proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Give Streamlit a moment to boot
time.sleep(3)

# Create public tunnel
public_url = ngrok.connect(addr=port)
print("Streamlit is running! Access it at:", public_url)


In [ ]:
# Simple Historical Signal Scanner - Scan a specific pair or all pairs for historical signals

import asyncio
import nest_asyncio
import pandas as pd
import warnings
import logging

# Enable nested event loops for Jupyter
nest_asyncio.apply()
warnings.filterwarnings('ignore')

# Import our scanner module
from direct_test import (
    HistoricalSignalScanner, 
    save_signals_to_csv,
    analyze_signals,
    filter_signals
)

print("Historical Signal Scanner Ready!")
print("=" * 50)

# =============================================================================
# MAIN SCANNING FUNCTION
# =============================================================================

async def scan_historical_signals(symbol=None, exchange="binance_futures", timeframe="1d", 
                                 strategies=None, max_symbols=None):
    """
    Scan for historical signals
    
    Args:
        symbol: Specific symbol to scan (e.g., "BTCUSDT") or None to scan all
        exchange: Exchange to use (default: "binance_futures")
        timeframe: Timeframe to analyze (default: "1d")
        strategies: List of strategies or None for all available
        max_symbols: Max number of symbols to scan when symbol=None (default: 50)
    
    Returns:
        pandas.DataFrame: All detected signals
    """
    
    # Default strategies if none provided
    if strategies is None:
        strategies = ['hbs_breakout', 'confluence', 'wedge_breakout', 'channel_breakout', 
                     'consolidation_breakout', 'sma50_breakout']
    
    # Default max symbols
    if max_symbols is None:
        max_symbols = 50
    
    scanner = HistoricalSignalScanner(exchange, timeframe)
    
    try:
        await scanner.initialize()
        print(f"Initialized {exchange} scanner for {timeframe} timeframe")
        
        if symbol:
            # Scan specific symbol
            print(f"Scanning {symbol}...")
            results = await scanner.scan_symbol_historical(symbol, strategies)
            df = scanner.create_signals_dataframe(results)
            
            if not df.empty:
                filename = f"{symbol}_{exchange}_{timeframe}_signals"
                save_signals_to_csv(df, filename)
                print(f"\nFound {len(df)} signals for {symbol}")
                analyze_signals(df)
            else:
                print(f"No signals found for {symbol}")
                
        else:
            # Scan all symbols
            print(f"Getting symbols from {exchange}...")
            symbols = await scanner.get_all_symbols(limit=max_symbols)
            print(f"Scanning {len(symbols)} symbols...")
            
            results = await scanner.scan_multiple_symbols(symbols, strategies)
            df = scanner.create_signals_dataframe(results)
            
            if not df.empty:
                filename = f"all_symbols_{exchange}_{timeframe}_{len(symbols)}pairs_signals"
                save_signals_to_csv(df, filename)
                print(f"\nFound {len(df)} total signals across {len(symbols)} symbols")
                analyze_signals(df)
                
                # Show top symbols by signal count
                print(f"\nTop symbols by signal count:")
                symbol_counts = df['symbol'].value_counts().head(10)
                for sym, count in symbol_counts.items():
                    print(f"  {sym}: {count} signals")
            else:
                print(f"No signals found across {len(symbols)} symbols")
        
        return df
        
    finally:
        await scanner.close()

# =============================================================================
# QUICK COMMANDS
# =============================================================================

async def scan_pair(symbol, exchange="binance_futures", timeframe="1d"):
    """Quick scan for a specific trading pair"""
    return await scan_historical_signals(symbol=symbol, exchange=exchange, timeframe=timeframe)

async def scan_all_pairs(exchange="binance_futures", timeframe="1d", max_pairs=50):
    """Quick scan for all trading pairs"""
    return await scan_historical_signals(symbol=None, exchange=exchange, 
                                        timeframe=timeframe, max_symbols=max_pairs)


df = await scan_historical_signals(exchange="mexc_spot", timeframe="2d",
    symbol="NOS_USDT",
    strategies=['hbs_breakout']
)

In [ ]:
#ASCII CHART + Testing mexc 1w candles

import requests
import datetime as dt

MEXC_URL = "https://api.mexc.com/api/v3/klines"
SYMBOL = "STBLUSDT"
INTERVAL = "1W"         # MEXC uses uppercase W for weekly klines
LIMIT = 50              # ask for more than 5 so we can always slice down

HEADERS = {"User-Agent": "console-fetch/1.0 (+https://example.com)"}

def fetch_mexc_weekly(symbol=SYMBOL, limit=LIMIT):
    params = {
        "symbol": symbol,
        "interval": INTERVAL,
        "limit": limit
    }
    r = requests.get(MEXC_URL, params=params, headers=HEADERS, timeout=20)
    r.raise_for_status()
    data = r.json()
    if not isinstance(data, list) or not data:
        raise RuntimeError(f"Unexpected response: {data}")

    # Each kline: [openTime, open, high, low, close, volume, closeTime, ...]
    candles = []
    for row in data:
        open_time_ms = int(row[0])
        candles.append({
            "time": dt.datetime.utcfromtimestamp(open_time_ms / 1000.0),
            "open": float(row[1]),
            "high": float(row[2]),
            "low":  float(row[3]),
            "close": float(row[4]),
            "volume": float(row[5]),
        })
    return candles

def ascii_candles(candles, width=40):
    """
    Render simple ASCII candles for a small set of bars.
    For each candle, draw a range [low..high] as a line, and mark:
      O = open, C = close. Up candles also show '+' at close; down show '-'.
    The scale is shared across the provided candles.
    """
    if not candles:
        return

    lo = min(c["low"] for c in candles)
    hi = max(c["high"] for c in candles)
    if hi == lo:
        hi = lo + 1e-9  # avoid zero range

    def pos(price):
        # map a price into [0..width-1]
        return int(round((price - lo) / (hi - lo) * (width - 1)))

    lines = []
    for c in candles:
        line = [" "] * width
        low_p, high_p = pos(c["low"]), pos(c["high"])
        open_p, close_p = pos(c["open"]), pos(c["close"])
        # draw range
        for i in range(low_p, high_p + 1):
            line[i] = "─"
        # mark open/close
        line[open_p] = "O"
        if close_p == open_p:
            # nudge to show both if equal
            close_p = min(width - 1, close_p + 1)
        line[close_p] = "C"
        # up/down marker at close
        line[close_p] = "+" if c["close"] >= c["open"] else "-"
        lines.append("".join(line))
    return lines, lo, hi

def fmt_num(x):
    # compact formatting for prices
    if x >= 1000:
        return f"{x:,.0f}"
    return f"{x:,.2f}"

def main():
    candles = fetch_mexc_weekly()
    last14 = candles[-14:]  # last 5 weekly bars

    # Print a small info table
    print("Last 5 weekly candles (MEXC spot, BTCUSDT, 1W):")
    print("Date (UTC)     |     Open       High        Low       Close        Vol")
    print("-" * 74)
    for c in last14:
        print(f"{c['time'].date()} | {fmt_num(c['open']).rjust(10)}  {fmt_num(c['high']).rjust(10)}  "
              f"{fmt_num(c['low']).rjust(10)}  {fmt_num(c['close']).rjust(10)}  {fmt_num(c['volume']).rjust(10)}")

    print("\nASCII chart (shared scale across the 14 bars):")
    chart, lo, hi = ascii_candles(last14, width=48)
    # Put labels and lines together
    for i, (c, line) in enumerate(zip(last14, chart)):
        label = c["time"].strftime("%Y-%m-%d")
        print(f"{label} | {line} | O={fmt_num(c['open'])} C={fmt_num(c['close'])}")

    print(f"\nScale: low={fmt_num(lo)}  high={fmt_num(hi)}")

if __name__ == "__main__":
    main()


In [ ]:
# ===== Breakout monitor for current & previous bar (HBS-aligned) =====
import pandas as pd
import numpy as np
import requests, sys

# Make sure your Project path is on sys.path (adjust if needed)
sys.path.append('/home/jovyan/work/Crypto/sevenfigures-bot/hbs_2025/Project/Project')

# ---- Tunables (match your detector) ----
HA_MA_LENGTH = 13
AMA_PERIOD   = 2
AMA_FAST     = 1
AMA_SLOW     = 15
JS_SMOOTH    = 13
JS_POWER     = 5

# ---------- Data fetch ----------
def fetch_bybit_data(symbol, timeframe="1w", limit=100):
    symbol_clean = symbol.replace("USDT", "")
    url = "https://api.bybit.com/v5/market/kline"
    params = {"category":"spot","symbol":f"{symbol_clean}USDT","interval":"W","limit":limit}
    r = requests.get(url, params=params, timeout=10)
    data = r.json()
    if data.get("retCode") != 0:
        print(f"Bybit API error for {symbol}: {data.get('retMsg')}")
        return None
    rows = [{
        "timestamp": pd.to_datetime(int(k[0]), unit="ms"),
        "open": float(k[1]), "high": float(k[2]),
        "low":  float(k[3]), "close": float(k[4]),
        "volume": float(k[5]),
    } for k in reversed(data["result"]["list"])]
    df = pd.DataFrame(rows).set_index("timestamp")
    return df[["open","high","low","close","volume"]]

# ---------- Helpers (mirror your detector) ----------
def ema(s: pd.Series, span: int) -> pd.Series:
    return s.ewm(span=span, adjust=False).mean()

def wma(s: pd.Series, length: int) -> pd.Series:
    weights = np.arange(1, length + 1, dtype=float)
    return s.rolling(length, min_periods=1).apply(
        lambda x: np.dot(x, weights[-len(x):]) / weights[-len(x):].sum(),
        raw=True
    )

def atr_wilder(h: pd.Series, l: pd.Series, c: pd.Series, length: int = 7) -> pd.Series:
    prev_close = c.shift(1)
    tr = pd.concat([(h - l).abs(), (h - prev_close).abs(), (l - prev_close).abs()], axis=1).max(axis=1)
    return tr.ewm(alpha=1/length, adjust=False).mean()

def ama(series, period=2, period_fast=2, period_slow=30, epsilon=1e-10):
    n = period + 1
    src = np.asarray(series, dtype=float)
    hh = pd.Series(src).rolling(window=n, min_periods=1).max().values
    ll = pd.Series(src).rolling(window=n, min_periods=1).min().values
    mltp = np.where((hh - ll) != 0, np.abs(2 * src - ll - hh) / (hh - ll + epsilon), 0.0)
    sc_fastest = 2 / (period_fast + 1)
    sc_slowest = 2 / (period_slow + 1)
    sc = (mltp * (sc_fastest - sc_slowest) + sc_slowest) ** 2
    sc = np.nan_to_num(sc, nan=0.0, posinf=0.0, neginf=0.0)
    out = np.zeros_like(src)
    out[:period] = src[:period]
    for i in range(period, len(src)):
        out[i] = out[i - 1] + sc[i] * (src[i] - out[i - 1])
    return pd.Series(out)

def jsmooth(src, smooth, power):
    s = np.asarray(src, dtype=float)
    beta = 0.45 * (smooth - 1) / (0.45 * (smooth - 1) + 2)
    alpha = beta ** power
    n = len(s)
    jma = np.zeros(n)
    e0 = np.zeros(n); e1 = np.zeros(n); e2 = np.zeros(n)
    e0[0] = s[0]; e1[0] = 0.0; e2[0] = 0.0; jma[0] = s[0]
    for i in range(1, n):
        e0[i] = (1 - alpha) * s[i] + alpha * e0[i - 1]
        e1[i] = (s[i] - e0[i]) * (1 - beta) + beta * e1[i - 1]
        e2[i] = (e0[i] - jma[i - 1]) * ((1 - alpha) ** 2) + (alpha ** 2) * e2[i - 1]
        jma[i] = jma[i - 1] + e2[i]
    return pd.Series(jma)

def pivot_calc(osc: list[float], LBL=2, LBR=2, highlow='high') -> list[float]:
    n = len(osc)
    piv = [np.nan]*n
    if n == 0: return piv
    for center in range(LBL + LBR, n):
        ref_index = center - LBR
        ref = osc[ref_index]
        left, right = ref_index - LBL, ref_index + LBR
        if left < 0 or right >= n: continue
        ok = True
        for j in range(left, right + 1):
            if j == ref_index: continue
            if highlow == 'high':
                if osc[j] >= ref: ok = False; break
            else:
                if osc[j] <= ref: ok = False; break
        if ok: piv[ref_index] = ref
    return piv

# ---------- Build the same columns your (updated) detector uses ----------
def build_detector_columns(df: pd.DataFrame) -> pd.DataFrame:
    d = df.copy()
    d["atr_7"] = atr_wilder(d["high"], d["low"], d["close"], 7)

    # lac and AMA(2, fast=1, slow=15)
    d["lac"] = (d["open"] + d["close"])/2 + (((d["close"] - d["open"]) / (d["high"] - d["low"] + 1e-6)) * ((d["close"] - d["open"]).abs()/2))
    d["habclose"] = ama(d["lac"].values, period=AMA_PERIOD, period_fast=AMA_FAST, period_slow=AMA_SLOW).values

    # recursive habopen
    ho = np.zeros(len(d), dtype=float)
    ho[0] = (d["open"].iloc[0] + d["close"].iloc[0]) / 2.0
    for i in range(1, len(d)):
        ho[i] = (ho[i-1] + d["habclose"].iloc[i-1]) / 2.0
    d["habopen"] = ho

    # HA high/low
    d["habhigh"] = pd.concat([d["high"], d["habopen"], d["habclose"]], axis=1).max(axis=1)
    d["hablow"]  = pd.concat([d["low"],  d["habopen"], d["habclose"]], axis=1).min(axis=1)

    # jsmooth and s_habhigh/s_hablow
    d["jsmooth_habhigh"] = jsmooth(d["habhigh"].values, JS_SMOOTH, JS_POWER).values
    d["jsmooth_hablow"]  = jsmooth(d["hablow"].values,  JS_SMOOTH, JS_POWER).values

    ema_high = ema(pd.Series(d["jsmooth_habhigh"], index=d.index), span=HA_MA_LENGTH)
    wma_high = wma(pd.Series(d["jsmooth_habhigh"], index=d.index), length=HA_MA_LENGTH)
    d["s_habhigh"] = (ema_high + wma_high) / 2
    d["s_hablow"]  = ema(pd.Series(d["jsmooth_hablow"], index=d.index), span=HA_MA_LENGTH)

    # Pivot-high level (+LBR shift/ffill)
    ph = pivot_calc(d["high"].astype(float).tolist(), 2, 2, "high")
    d["ph"] = pd.Series(ph, index=d.index).shift(2)
    d["ph_range"] = d["ph"].ffill()

    # Levels & booleans
    d["level_sh"] = d["s_habhigh"] + 0.1*d["atr_7"]             # breakout_condition level
    d["level_ph"] = d["ph_range"]   + 0.3*d["atr_7"]             # UpWeGo level

    d["breakout_condition"] = d["close"] > d["level_sh"]
    d["breakout_prev"]      = d["breakout_condition"].shift(1).fillna(False)

    d["breakup"] = d["close"] >= d["level_ph"]                   # pivot breakout
    bu1 = d["breakup"].shift(1).fillna(False)
    bu2 = d["breakup"].shift(2).fillna(False)
    pivot_updated = d["ph_range"].ne(d["ph_range"].shift(1)).fillna(False)
    d["upwego"] = d["breakup"] & ((d["breakup"] & ~bu1) | (bu1 & ~bu2) | pivot_updated)

    return d

# ---------- Monitor two bars: -2 (closed) and -1 (current) ----------
def monitor_breakout(symbol: str):
    df = fetch_bybit_data(symbol, "1w", 100)
    if df is None or len(df) < 10:
        print("No data.")
        return
    d = build_detector_columns(df)

    i_prev = len(d) - 2  # -2
    i_curr = len(d) - 1  # -1

    def row(i):
        return {
            "time": d.index[i],
            "close": float(d["close"].iloc[i]),
            "s_habhigh": float(d["s_habhigh"].iloc[i]),
            "atr7": float(d["atr_7"].iloc[i]),
            "level_sh": float(d["level_sh"].iloc[i]),
            "breakout_cond": bool(d["breakout_condition"].iloc[i]),
            "breakout_prev": bool(d["breakout_prev"].iloc[i]),
            "PH_range": float(d["ph_range"].iloc[i]) if not pd.isna(d["ph_range"].iloc[i]) else None,
            "level_ph": float(d["level_ph"].iloc[i]) if not pd.isna(d["level_ph"].iloc[i]) else None,
            "breakup": bool(d["breakup"].iloc[i]),
            "upwego": bool(d["upwego"].iloc[i]),
        }

    print(f"\n=== {symbol} @ 1W  (monitoring -2 and -1) ===")
    print(f"Settings: AMA(period={AMA_PERIOD}, fast={AMA_FAST}, slow={AMA_SLOW}), "
          f"HA_MA_LENGTH={HA_MA_LENGTH}, JS(smooth={JS_SMOOTH}, power={JS_POWER})")
    print("Last 4 bars:")
    print(df.tail(4), "\n")

    R_prev = row(i_prev)
    R_curr = row(i_curr)

    def p(R, tag):
        print(f"[{tag}] {R['time']}")
        print(f"  close={R['close']:.8f}")
        print(f"  s_habhigh={R['s_habhigh']:.8f}  atr7={R['atr7']:.8f}")
        print(f"  level_sh=s_habhigh+0.1*atr7 = {R['level_sh']:.8f}")
        print(f"  breakout_condition  = {R['breakout_cond']}   (prev={R['breakout_prev']})")
        print(f"  ph_range={R['PH_range']}  level_ph={R['level_ph']}")
        print(f"  breakup(pivot>=)    = {R['breakup']}  upwego={R['upwego']}\n")

    p(R_prev, "BAR -2 (last closed)")
    p(R_curr, "BAR -1 (current)")

    # Support set @ -2 (mirrors detector)
    conds_prev = {
        "upwego": d["upwego"].iloc[i_prev],
        "atr_trend": (d["atr_7"].iloc[i_prev] - d["atr_7"].iloc[i_prev-1]) >= 0.01*max(1e-12, d["atr_7"].iloc[i_prev-1]),
        "ma_bull": ema(d["habclose"],5).iloc[i_prev] > ema(d["habopen"],10).iloc[i_prev],
        "ha_momentum": d["habclose"].iloc[i_prev] > d["habopen"].iloc[i_prev],
        "flagup": (d["high"].iloc[i_prev] > d["high"].iloc[i_prev-1]) and \
                  ((d["high"].iloc[i_prev]-d["close"].iloc[i_prev]) < (d["close"].iloc[i_prev]-d["low"].iloc[i_prev])),
    }
    print("Support set @ -2:", {k: bool(v) for k,v in conds_prev.items()})
    print("Breakout edge @ -2 (False->True)?", bool(d["breakout_condition"].iloc[i_prev] and not d["breakout_prev"].iloc[i_prev]))

# ---- run ----
monitor_breakout("ETHFIUSDT")
# Try also: monitor_breakout("SOSOUSDT"); monitor_breakout("ATHUSDT")
